In [24]:
import requests
import re
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import json

In [6]:
# soup = BeautifulSoup(html, "lxml")

In [7]:
# soup = BeautifulSoup(html, features="xml")

In [8]:

# # extract links from a page, credit: https://pythonspot.com/extract-links-from-webpage-beautifulsoup/
# # I had to <pip3 install lxml> to get this to work
# def getLinks(url):
#     html_page = urlopen(url)
#     soup = BeautifulSoup(html_page, features="lxml")
#     links = []

#     for link in soup.findAll('a', attrs={'href': re.compile("^/Movie Scripts/")}):
#         links.append(link.get('href'))

#     return links

# # this page is a list of all the movies IMSDb has on file for the Sci-Fi genre
# print( getLinks("https://imsdb.com/genre/Sci-Fi") )

# # Now, we need to wade through list of links generated by getLinks and extract
# # the urls for just the scripts, like is down below:

# # this is just the script of a movie
# URL = "https://imsdb.com/scripts/12-Monkeys.html"
# page = requests.get(URL)

# soup = BeautifulSoup(page.content, "html.parser")
# script = soup.find('pre')

# # print(script.text.strip())

In [9]:
!python scrape_scripts.py

In [10]:
#print(script.text.strip())

In [11]:
# blackpanther = "https://imsdb.com/scripts/Black-Panther.html"
# page_bp = requests.get(blackpanther)

# soup_bp = BeautifulSoup(page_bp.content, "html.parser")
# script_bp = soup_bp.find('pre')

In [12]:
# print(script_bp.text.strip())

BLACK PANTHER

                         

                          
                           Written by 

                 Ryan Coogler & Joe Robert Cole






          EXT. DEEP SPACE

          A dark screen is lit up by twinkling stars .

                          SON
           Baba?

                          FATHER
           Yes, my son?

                          SON
           Tell me a story .

                          FATHER
           Which one?

                          SON
           The story of home .
          A meteorite drifts into frame , heading towards tiny Earth off
          in the distance.

                          FATHER
           Millions of years ago , a meteorite
           made of vibranium, the strongest
           substance in the universe struck
           the continent of Africa affecting
           the plant life around it.

          The meteorite hits Africa and we see plant life and animals
          affected by vibranium.

               

In [13]:
# !pip install convokit

In [14]:
# from convokit import Corpus, download

In [15]:
# corpus = Corpus(filename=download("movie-corpus"))

In [25]:
from pandas.io.json import json_normalize

In [26]:
path_to_file = "scripts.txt"

In [27]:
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 4539732 characters


In [28]:
print(text[:250])

Quentin Tarantino's

                         R E S E R V O I R   D O G S








       October 22, 1990

                               -----------------


       This movie is dedicated to these following sources of

       inspi


In [ ]:
# temp = open("movie-corpus/corpus.json")
# datum = json.load(temp)

In [ ]:
# corpus = json_normalize(datum)

In [ ]:
# corpus

In [ ]:
# script_bp = script_bp.text.strip()

In [31]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing

import numpy as np
import os
import time

In [32]:
vocab = sorted(set(text.strip()))
print('{} unique characters'.format(len(vocab)))

96 unique characters


In [33]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [34]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab))

In [35]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[62, 63, 64, 65, 66, 67, 68], [85, 86, 87]]>

In [36]:
chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True)

In [37]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

In [38]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [39]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [40]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(4539732,), dtype=int64, numpy=array([49, 82, 66, ..., 53, 50, 37])>

In [41]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [42]:
for ids in ids_dataset.take(13):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

Q
u
e
n
t
i
n
 
T
a
r
a
n


In [43]:
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

In [44]:
examples_per_epoch

44947

In [45]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'Q' b'u' b'e' b'n' b't' b'i' b'n' b' ' b'T' b'a' b'r' b'a' b'n' b't'
 b'i' b'n' b'o' b"'" b's' b'\r' b'\n' b'\r' b'\n' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b' ' b'R' b' ' b'E' b' ' b'S' b' ' b'E' b' '
 b'R' b' ' b'V' b' ' b'O' b' ' b'I' b' ' b'R' b' ' b' ' b' ' b'D' b' '
 b'O' b' ' b'G' b' ' b'S' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n'
 b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b'\r' b'\n' b' ' b' '
 b' ' b' ' b' ' b' ' b' ' b'O'], shape=(101,), dtype=string)


In [46]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b"Quentin Tarantino's\r\n\r\n                         R E S E R V O I R   D O G S\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n       O"
b'ctober 22, 1990\r\n\r\n                               -----------------\r\n\r\n\r\n       This movie is dedicat'
b'ed to these following sources of\r\n\r\n       inspiration:\r\n\r\n\r\n                               TIMOTHY C'
b'AREY\r\n\r\n                                ROGER CORMAN\r\n\r\n                                ANDRE DeTOTH\r'
b'\n\r\n                                CHOW YUEN FAT\r\n\r\n                               JEAN LUC GODDARD\r\n'


In [47]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [48]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [49]:
dataset = sequences.map(split_input_target)

In [50]:
for input_example, target_example in  dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b"Quentin Tarantino's\r\n\r\n                         R E S E R V O I R   D O G S\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n       "
Target: b"uentin Tarantino's\r\n\r\n                         R E S E R V O I R   D O G S\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n       O"


In [51]:
# Batch size
BATCH_SIZE = 64

BUFFER_SIZE = 1000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<PrefetchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [52]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(rnn_units,
                                   return_sequences=True, 
                                   return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, states=None, return_state=False, training=False):
        x = inputs
        x = self.embedding(x, training=training)
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else: 
            return x

In [53]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [54]:
model = MyModel(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

In [55]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 97) # (batch_size, sequence_length, vocab_size)


In [56]:
model.summary()

Model: "my_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  24832     
                                                                 
 gru (GRU)                   multiple                  3938304   
                                                                 
 dense (Dense)               multiple                  99425     
                                                                 
Total params: 4,062,561
Trainable params: 4,062,561
Non-trainable params: 0
_________________________________________________________________


In [57]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [58]:
sampled_indices

array([10, 13, 78, 86, 51, 72, 27, 54, 23, 63, 45, 91,  4, 79,  8, 24, 66,
       66, 20, 46,  3, 61, 29, 51, 83, 67, 96, 20,  2, 40, 55, 35, 19, 15,
       82, 67, 79, 67,  7, 62, 19, 29, 57, 59, 46,  3, 59, 84, 39, 38, 26,
       58, 72, 95, 26, 74, 82, 25, 83, 64, 34,  0, 90, 74, 63, 78, 92, 16,
       93, 45, 19, 11, 73, 86, 38, 25, 56, 16, 28, 32, 57, 73, 86,  6, 41,
       95, 80, 92, 34, 15, 93, 45, 68, 68, 44,  0, 56, 59, 72, 29])

In [59]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'back over him and takes his hand.\r\n\r\n                              MR. WHITE\r\n                 I ain'

Next Char Predictions:
 b'&)qySk9V5bM\xc2\x9c r$6ee2N\r^;Svf\xc3\xa22\nHWC1-ufrf#a1;Y\\N\r\\wGF8Zk\xc2\xbd8mu7vcB[UNK]\xc2\x98mbq\xc2\x9d.\xc2\xabM1\'lyF7X.:?Yly"I\xc2\xbds\xc2\x9dB-\xc2\xabMggL[UNK]X\\k;'


In [60]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [61]:
example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

Prediction shape:  (64, 100, 97)  # (batch_size, sequence_length, vocab_size)
Mean loss:         4.571388


In [62]:
tf.exp(mean_loss).numpy()

96.678185

In [63]:
model.compile(optimizer='adam', loss=loss)

In [64]:
# Directory where the checkpoints will be saved
checkpoint_dir = './deneme2'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [65]:
EPOCHS = 1

In [66]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

702/702 [==============================] - 2553s 4s/step - loss: 1.6429


In [67]:
class OneStep(tf.keras.Model):
    def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
        super().__init__()
        self.temperature=temperature
        self.model = model
        self.chars_from_ids = chars_from_ids
        self.ids_from_chars = ids_from_chars

    # Create a mask to prevent "" or "[UNK]" from being generated.
        skip_ids = tf.constant([[0], [0]], dtype=tf.int64)
        tmp1 = tf.reshape(skip_ids, shape=(-1,))
        uniques, idx, counts = tf.unique_with_counts(tmp1)
        uniques_ids = tf.expand_dims(uniques, axis=1)
        
        sparse_mask = tf.SparseTensor(
    # Put a -inf at each bad index.
                values=[-float('inf')] * len(uniques_ids),
                indices=uniques_ids,
    # Match the shape to the vocabulary
                dense_shape=[len(ids_from_chars.get_vocabulary())])
        print(sparse_mask)
        
#         #skip_ids = self.ids_from_chars(['','[UNK]'])[:, None]
#         sparse_mask = tf.SparseTensor(
#         # Put a -inf at each bad index.
#             values=[-float('inf')]*len(skip_ids),
#             indices = skip_ids,
#         # Match the shape to the vocabulary
#             dense_shape=[len(ids_from_chars.get_vocabulary())]) 

        self.prediction_mask = tf.sparse.to_dense(sparse_mask)

    @tf.function
    def generate_one_step(self, inputs, states=None):
    # Convert strings to token IDs.
        input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
        input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Run the model.
    # predicted_logits.shape is [batch, char, next_char_logits] 
        predicted_logits, states =  self.model(inputs=input_ids, states=states, 
                                          return_state=True)
    # Only use the last prediction.
        predicted_logits = predicted_logits[:, -1, :]
        predicted_logits = predicted_logits/self.temperature
    # Apply the prediction mask: prevent "" or "[UNK]" from being generated.
        predicted_logits = predicted_logits + self.prediction_mask

    # Sample the output logits to generate token IDs.
        predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
        predicted_ids = tf.squeeze(predicted_ids, axis=-1)
    
    # Convert from token ids to characters
        predicted_chars = self.chars_from_ids(predicted_ids)

    # Return the characters and model state.
        return predicted_chars, states

In [68]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

SparseTensor(indices=tf.Tensor([[0]], shape=(1, 1), dtype=int64), values=tf.Tensor([-inf], shape=(1,), dtype=float32), dense_shape=tf.Tensor([97], shape=(1,), dtype=int64))


In [69]:
start = time.time()
states = None
next_char = tf.constant(['N\'JOBU:'])
result = [next_char]

for n in range(10000):
    next_char, states = one_step_model.generate_one_step(next_char, states=states)
    result.append(next_char)

result = tf.strings.join(result)
end = time.time()

print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)

print(f"\nRun time: {end - start}")

N'JOBU:

                     There -- them on the handy one the to wer.
mecuuse it it, russusing of a nisty behind seens doleden.

                           SIDNNEY
                Hindecter.!.Hunfel costing tow.

                                           (E7TING

I fight is a Don't stepice.


SEVE IN THE RISHO DRE SIDNEY ANGELE

                    us fromated it to the looks to stage...

The steve now over Haccuper's weal tho ebed ang rony never:

                          KELLO
             Goldon.... gurls to y.a.  I
                   upe for here.  Is stu poony hellace Nolan angrow
of
             hel'dny Crymondly, beent to the hostion,
lettle vey walk indo men every the rearent of stalle -
whe moremt encers hows he hes tablle, becrush the come hurner witeching up glanch is.

                           my · walksy.  Where ace, Amaut.

                            (SIDNEY
            No.  In you in belid; Mr. Hon eyes ho --giture.

She varps op her
an of the fact - ever him aga

In [70]:
print(type(result))

<class 'tensorflow.python.framework.ops.EagerTensor'>


next steps: find nonsense words, count num nonsense/normal words --> success metric

Code used: [the original script](https://github.com/tarik3333/Movie-script-gen/blob/main/Untitled9.ipynb) and [this answer](https://stackoverflow.com/questions/68939916/tensorflow-text-generation-rnn-example-failing-on-tf-2-6-tf-sparse-to-dense)